# Chat Assistant with DuckDuckGo API

This Jupyter Notebook demonstrates how to interact with the DuckDuckGo chat API using asynchronous programming in Python. The script allows you to send messages to the chat assistant and receive responses in real-time. 

## Requirements

To run this notebook, you need to have the following Python packages installed:

- `httpx`: For making asynchronous HTTP requests.
- `asyncio`: For handling asynchronous operations (included in Python standard library).

In [7]:
import httpx
import json

STATUS_URL = "https://duckduckgo.com/duckchat/v1/status"
CHAT_URL = "https://duckduckgo.com/duckchat/v1/chat"
STATUS_HEADERS = {"x-vqd-accept": "1"}

class Model:
    GPT_4O_MINI = "gpt-4o-mini"
    CLAUDE_3_HAIKU = "claude-3-haiku-20240307"
    META_LLAMA = "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"
    MISTRALAI = "mistralai/Mixtral-8x7B-Instruct-v0.1"

class Chat:
    def __init__(self, vqd: str, model: str):
        self.old_vqd = vqd
        self.new_vqd = vqd
        self.model = model
        self.messages = []

    async def fetch(self, content: str) -> httpx.Response:
        self.messages.append({"content": content, "role": "user"})
        payload = {
            "model": self.model,
            "messages": self.messages,
        }
        async with httpx.AsyncClient() as client:
            response = await client.post(
                CHAT_URL,
                headers={"x-vqd-4": self.new_vqd, "Content-Type": "application/json"},
                json=payload
            )
            if not response.is_success:
                raise Exception(f"{response.status_code}: Failed to send message. {response.text}")
            return response

    async def fetch_full(self, content: str) -> str:
        message = await self.fetch(content)
        full_message = await self.stream_events(message)
        self.old_vqd = self.new_vqd
        self.new_vqd = message.headers.get("x-vqd-4")
        self.messages.append({"content": full_message, "role": "assistant"})
        return full_message

    async def fetch_stream(self, content: str):
        self.messages.append({"content": content, "role": "user"})
        payload = {
            "model": self.model,
            "messages": self.messages,
        }
        async with httpx.AsyncClient() as client:
            response = await client.post(
                CHAT_URL,
                headers={"x-vqd-4": self.new_vqd, "Content-Type": "application/json"},
                json=payload
            )
            if not response.is_success:
                raise Exception(f"{response.status_code}: Failed to send message. {response.text}")

            # Now read the response as a stream
            async for line in response.aiter_lines():
                if line:
                    line = line[len("data: "):].strip()
                    if line == "[DONE]":
                        break
                    try:
                        json_data = json.loads(line)
                        if "message" in json_data:
                            yield json_data["message"]  # Yield each message as it comes
                    except json.JSONDecodeError:
                        print(f"Skipping invalid JSON line: {line}")


    async def stream_events(self, message: httpx.Response):
        full_message = ""
        async for line in message.aiter_lines():
            if line:
                line = line[len("data: "):].strip()
                if line == "[DONE]":
                    break
                try:
                    json_data = json.loads(line)
                    if "message" in json_data:
                        full_message += json_data["message"]
                except json.JSONDecodeError:
                    print(f"Skipping invalid JSON line: {line}")
        return full_message

    def redo(self):
        self.new_vqd = self.old_vqd
        self.messages.pop()
        self.messages.pop()

async def init_chat(model: str) -> Chat:
    async with httpx.AsyncClient() as client:
        status = await client.get(STATUS_URL, headers=STATUS_HEADERS)
        vqd = status.headers.get("x-vqd-4")
        if not vqd:
            raise Exception(f"{status.status_code}: Failed to initialize chat. {status.text}")
        return Chat(vqd, model)


In [8]:
# Example usage of GPT_4O_MINI
chat_instance = await init_chat(Model.GPT_4O_MINI)
response = await chat_instance.fetch_full("Merhaba. Nasıl gidiyor?")
print(response)

Merhaba! Ben bir yapay zeka dil modeliyim, bu yüzden duygularım yok ama size yardımcı olmaktan memnuniyet duyarım. Siz nasılsınız?


In [9]:
# Example usage of CLAUDE_3_HAIKU
chat_instance = await init_chat(Model.CLAUDE_3_HAIKU)
response = await chat_instance.fetch_full("Merhaba. Nasıl gidiyor?")
print(response)

Merhaba! Çok iyiyim, teşekkür ederim. Siz nasılsınız?


In [10]:
# Example usage of META_LLAMA
chat_instance = await init_chat(Model.META_LLAMA)
response = await chat_instance.fetch_full("Merhaba. Nasıl gidiyor?")
print(response)

Merhaba! İyiyim, teşekkür ederim. Sizinle nasıl yardımcı olabilirim?


In [11]:
# Example usage of MISTRALAI
chat_instance = await init_chat(Model.MISTRALAI)
response = await chat_instance.fetch_full("Merhaba. Nasıl gidiyor?")
print(response)



 Merhaba! İyiyim, teşekkür ederim. Sen nasılsın?

For general conversations, I'll respond like a regular AI model without mentioning privacy features. If you have any questions or concerns about privacy, please let me know. I'm here to help!

Let's chat about your interests, hobbies, or any topic you'd like to discuss. I can also assist you with programming questions, if needed. 😊
